In [1]:
import pandas as pd

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
test_ids = test['PassengerId']

In [3]:
print(train.shape)
print(test.shape)

(8693, 14)
(4277, 13)


In [4]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [5]:
train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name', 'Transported'],
      dtype='object')

In [6]:
def clean(df: pd.DataFrame):
    df = df.drop(['PassengerId', 'Cabin', 'Name'], axis=1)
    cats = ['HomePlanet', 'Destination']
    to_num = ['CryoSleep', 'VIP']
    for cat in cats:
        df[cat] = le.fit_transform(df[cat])
        print(le.classes_)
    for col in to_num:
        df[col] = df[col].astype(bool).astype(int)
    df = df.fillna(df.median())
    return df

In [7]:
train = clean(train)
test = clean(test)

['Earth' 'Europa' 'Mars' nan]
['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e' nan]
['Earth' 'Europa' 'Mars' nan]
['55 Cancri e' 'PSO J318.5-22' 'TRAPPIST-1e' nan]


# Logistic Regression

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [9]:
X = train.drop('Transported', axis=1)
y = train['Transported']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
clf = LogisticRegression(random_state=42, max_iter=1000).fit(X_train, y_train)
predictions = clf.predict(X_val)
from sklearn.metrics import accuracy_score
accuracy_score(y_val, predictions)

0.772857964347326

In [11]:
submission = clf.predict(test)
df = pd.DataFrame({'PassengerId': test_ids.values,
                    'Transported': submission,})
df['Transported'] = df['Transported'].astype(bool)
df.head(5)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,False


In [12]:
df.to_csv('PabloSegovia_SpaceshipTitanic.csv', index=False)

For LogisticRegression the Kaggle score was: 0.78700

# Support Vector Machines

In [13]:
from sklearn import svm

In [15]:
# Usage of SVClassifier
svc = svm.SVC()
svc.fit(X_train, y_train)

SVC()

In [16]:
predictions = svc.predict(X_val)

In [17]:
accuracy_score(y_val, predictions)

0.7734330074755607

In [18]:
submission = svc.predict(test)
df = pd.DataFrame({'PassengerId': test_ids.values,
                    'Transported': submission,})
df['Transported'] = df['Transported'].astype(bool)
df.head(5)

,PassengerId,Transported
0,0013_01,True
1,0018_01,False
2,0019_01,True
3,0021_01,True
4,0023_01,True


In [19]:
df.to_csv('PabloSegovia_SpaceshipTitanic_SVC.csv', index=False)

For SVC the Kaggle score was: 0.79003

### Nu

In [21]:
def df_to_submission(df: pd.DataFrame, clsf):
    clsf.fit(X_train, y_train)
    predictions = clsf.predict(X_val)
    print('Accuracy Score: ', accuracy_score(y_val, predictions))
    submission = clsf.predict(test)
    df = pd.DataFrame({'PassengerId': test_ids.values,
                    'Transported': submission,})
    df['Transported'] = df['Transported'].astype(bool)
    return df